# Análisis de resultados de varias carteras y posibilidad de representación gráfica

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../tablas/dataformodel.csv", usecols=['Price_d', 'Price_d+180',
                                                        'quantile_PER', 'var_quantile_PER','PER','var_PER',
                                                        'quantile_PBC', 'var_quantile_PBC','var_PBC',
                                                        'quantile_ROA', 'var_quantile_ROA','var_ROA',
                                                        'Etiqueta', 'Periodo','Ticker'])

df=df.replace([np.inf, -np.inf], np.nan)
for column in df.columns:
    df=df[df[column].notnull()]
df=df.reset_index(drop=True)

In [3]:
def resultado(row):
    if row['Prediction'] == 'Peor':
        return (row['Price_d'] - row['Price_d+180'])/row['Price_d']
    else:
        return (row['Price_d+180'] - row['Price_d'])/row['Price_d']

In [7]:
lista_periodos=df.Periodo.unique()[16:-1]
titulos = 7
inversion = 100
resumen = pd.DataFrame(columns=['Periodo','Rentabilidad'])
ratios = ['quantile_PER','var_quantile_PER','quantile_PBC','var_quantile_PBC']

for trim_seleccionado in lista_periodos:
    df_train = df[df.Periodo<trim_seleccionado]
    df_test = df[df.Periodo==trim_seleccionado]
    
    X_train=df_train[ratios].values
    
    y_train=pd.Series(df_train['Etiqueta'].values)
    
    
    X_test=df_test[ratios].values
    
    y_test=df_test['Etiqueta'].values
    
    clfk=KNeighborsClassifier(n_neighbors=5)
    clfk.fit(X_train,y_train)
    
    knGrid = GridSearchCV(clfk,cv=5,scoring="accuracy",param_grid={'n_neighbors':np.arange(1,20)})
    knGrid.fit(X_train,y_train)
    best_param=knGrid.best_params_.get('n_neighbors')
    
    clfk=KNeighborsClassifier(n_neighbors=best_param)
    clfk.fit(X_train,y_train)
    
    df_prob=pd.DataFrame({'Probabilidad':clfk.predict_proba(X_test)[:,0],'Prediction':clfk.predict(X_test),'Actual':y_test})
    df_prob.index=df_test.index

    best=df_prob.sort_values('Probabilidad').head(titulos)
    worst=df_prob.sort_values('Probabilidad').tail(titulos)
    cartera=best.append(worst)
    
    cartera = cartera.join(df_test)
    res_cartera=cartera[['Prediction','Price_d','Price_d+180']]
    
    res_cartera['Resultado']=res_cartera.apply(resultado,axis=1)*inversion
    
    
    resumen = resumen.append({'Periodo':trim_seleccionado,'Rentabilidad':res_cartera.Resultado.sum()}, ignore_index=True)
    

df_var_indice = pd.read_csv("../tablas/var_indice.csv")
resumen = resumen.merge(df_var_indice, 
                              left_on=('Periodo'),
                              right_on=('Periodo'), 
                              how='left')

resumen.drop(['Indice_d_100','Indice_d+180_100'], axis=1, inplace=True)

resumen
    

,Periodo,Rentabilidad,Rent_indice
0,2015Q3,11.504849,-9.133799
1,2015Q4,-80.213327,26.386224
2,2016Q1,16.537435,7.508099
3,2016Q2,-85.895429,18.713613
4,2016Q3,43.198886,2.854397
5,2016Q4,-151.473649,-16.455637
6,2017Q1,60.197158,-0.473791
7,2017Q2,149.982082,15.909931
8,2017Q3,-40.790239,9.942696
9,2017Q4,318.239678,12.941931


In [8]:
resumen.Rentabilidad.sum()

514.2873197181311

In [6]:
cartera

,Probabilidad,Prediction,Actual,Ticker,Periodo,Price_d,Price_d+180,Etiqueta,PER,var_PER,var_PBC,var_ROA,quantile_PER,quantile_PBC,quantile_ROA,var_quantile_PER,var_quantile_PBC,var_quantile_ROA
228,0.294118,Peor,Peor,CLR,2018Q4,44.26,29.67,Peor,83.263540,-0.286191,0.226356,0.617025,81.395349,76.744186,55.813953,18.895349,1.744186,-21.269380
258,0.294118,Peor,Peor,COG,2018Q4,24.34,16.47,Peor,38.106803,1.089517,-0.073260,-0.553103,53.488372,88.372093,97.674419,-21.511628,0.872093,8.091085
318,0.294118,Peor,Peor,CRR,2018Q4,4.08,1.56,Peor,-5.130000,0.594924,0.162729,-0.245342,27.906977,4.651163,9.302326,4.990310,-3.682171,5.135659
29,0.352941,Peor,Peor,AE,2018Q4,39.55,32.27,Peor,-43.352885,-2.873697,-0.039687,-1.470711,4.651163,37.209302,25.581395,-72.432171,8.042636,-20.251938
828,0.352941,Peor,Mejor,LNG,2018Q4,63.53,58.69,Mejor,243.594881,-0.010336,0.011538,0.009504,88.372093,90.697674,41.860465,-1.211240,-0.968992,6.443798
647,0.352941,Peor,Peor,HAL,2018Q4,29.97,18.79,Peor,39.313057,0.709263,0.185218,-0.339003,55.813953,81.395349,79.069767,-12.936047,0.145349,12.403101
468,0.352941,Peor,Peor,DVN,2018Q4,28.62,22.14,Peor,11.333420,-0.460467,0.229850,1.141675,37.209302,53.488372,93.023256,10.125969,-4.844961,-4.893411
1278,0.588235,Mejor,Peor,WMB,2018Q4,24.21,21.64,Peor,-51.447943,-4.502549,-0.254897,-1.216671,2.325581,62.790698,27.906977,-85.174419,14.874031,-9.593023
738,0.588235,Mejor,Mejor,INT,2018Q4,27.74,37.79,Mejor,62.485051,-0.120240,0.163561,0.215292,69.767442,25.581395,44.186047,11.434109,-5.668605,0.436047
1062,0.647059,Mejor,Peor,OXY,2018Q4,58.56,40.95,Peor,62.524827,-0.603764,0.041005,1.582358,72.093023,67.441860,62.790698,38.759690,2.858527,-33.042636
